In [221]:
#ToDo: add training graphs !
#ToDo: check microGrad and TinyGrad
#warmup_tokens and final_token 


In [222]:
import random
import math
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F # Cross-Entropy
import pytorch_lightning as pl
import os

from tqdm import tqdm # bar
import numpy as np
import torch.optim as optim # Adam optimizer
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.dataloader import DataLoader 


In [223]:
# model parameters

ndigit=2 #1,2,3,...
n_embd=128
vocab_size=10
block_size=3*ndigit
n_layer=2
n_heads=4

embd_pdrop = 0.1
resid_pdrop = 0.1
attn_pdrop = 0.1

DEVICE="cpu"

#training parameters

bs = 512
lr=6e-4
Max_Epochs=10


In [224]:
#dataset

In [225]:
def make_dataset():
    ret = []
    for i in range(10**ndigit):
        for j in range(10**ndigit):
            s = i+j
            ret.append([i//10, i%10, j//10, j%10, s//100, (s//10)%10, s%10])
    return ret
ds = make_dataset()
random.shuffle(ds)
ds = np.array(ds)
ds_X = ds[:, 0:6]
ds_Y = np.copy(ds[:, 1:])
ds_X_train, ds_X_test = ds_X[0:8000], ds_X[8000:]
ds_Y_train, ds_Y_test = ds_Y[0:8000], ds_Y[8000:]

#wrapper (ToDo: simplify & get rid of it.)
from torch.utils.data import Dataset
class AdditionDataset(Dataset):
    def __init__(self,X,Y):
        self.X=X
        self.Y=Y
    
    
    def __len__(self,):
        return self.X.shape[0]
    
    def __getitem__(self,idx):
        self.x = torch.tensor(self.X[idx], dtype=torch.long)
        self.y = torch.tensor(self.Y[idx], dtype=torch.long) # predict the next token in the sequence
        self.y[:ndigit*2-1] = -100 # we will only train in the output locations. -100 will mask loss to zero
        return self.x, self.y


In [226]:
AdditionDataset(ds_X_train,ds_Y_train)

train_dataset = AdditionDataset(ds_X_train,ds_Y_train)
test_dataset = AdditionDataset(ds_X_test,ds_Y_test)


In [227]:
print(ds_X[0])

[9 2 4 5 1 3]


In [228]:
print(ds_Y[0])

[2 4 5 1 3 7]


# torch.nn Module for GPT

In [229]:
class Embedder(nn.Module):
    """ simple aux class to embed data entry """  
    def __init__(self, block_size, vocab_size, n_embd):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, n_embd) # -> (BS,T,C)
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd)) # ->(1,T,C)
   
    def __call__(self,x):
        b, t = torch.Tensor(x).shape
        token_embeddings = self.tok_emb( torch.Tensor(x)) # each index maps to a (learnable) vector
        position_embeddings = self.pos_emb # each index maps to a (learnable) vector
        return token_embeddings+position_embeddings


class MultiHeadSelfAttention(nn.Module):
    def __init__(self):
        super(MultiHeadSelfAttention, self).__init__()
        assert n_embd % n_heads == 0
        self.proj_dim = n_embd // n_heads
        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)
        # output projection
        self.proj = nn.Linear(n_embd, n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size))
                                     .view(1, 1, block_size, block_size))      

        # 4 * embed_dim**2 weights
        
        self.query = nn.Linear(n_embd,n_embd)
        self.key = nn.Linear(n_embd,n_embd)
        self.value = nn.Linear(n_embd,n_embd)
        
        
    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, T, n_heads, self.proj_dim).transpose(1, 2) # (B, nh, T, hs)
        q = self.query(x).view(B, T, n_heads, self.proj_dim).transpose(1, 2) # (B, nh, T, hs)
        v = self.value(x).view(B, T, n_heads, self.proj_dim).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_drop(self.proj(y))
        return y

    
class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = MultiHeadSelfAttention()
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(resid_pdrop),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

    
class GPT(nn.Module):
    """  the full GPT language model, with a context size of block_size """

    def __init__(self):
        super().__init__()
        # embedder
        self.Embedder = Embedder(block_size,vocab_size,n_embd)
        # transformer
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        # dropout
        self.drop = nn.Dropout(embd_pdrop)
        # layer normalizations
        self.ln_f = nn.LayerNorm(n_embd)
        # head
        self.head = nn.Linear(n_embd, vocab_size, bias=False)
        self.apply(self._init_weights)
        
        print("number of parameters: %e", sum(p.numel() for p in self.parameters())) 
        
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)
        elif isinstance(module, GPT):
            torch.nn.init.normal_(module.Embedder.pos_emb, mean=0.0, std=0.02)
            
    def configure_optimizers(self, train_config):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('Embedder.pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer
        
    def forward(self,x,y=None):
        
        #cast numpy-> torch.Tensor
        x = torch.Tensor(x).to(DEVICE).long()
        if y is not None:
            y = torch.Tensor(y).to(DEVICE).long()
        
        x=self.Embedder(x)
        x = self.drop(x)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        
         # if we are given some desired targets <y>, also calculate the loss
        loss = None
        if y is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))

        return logits, loss

# Trainer 

In [230]:

class TrainerConfig: #configuration class
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_tokens = 375e6 # these two numbers come from the GPT-3 paper, but may not be good defaults elsewhere
    final_tokens = 260e9 # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_path = None
    num_workers = 0 # for DataLoader

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)    
    
class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config

        # take over whatever gpus are on the system
        self.device = 'cpu'

    def save_checkpoint(self):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        torch.save(raw_model.state_dict(), self.config.ckpt_path)

    def train(self):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)

        def run_epoch(loader, is_train):
            model.train(is_train)

            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (x, y) in pbar:

                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    logits, loss = model(x, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())
                
                if is_train:
                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # number of tokens processed this step (i.e. label is not -100)
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens))
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")

            if not is_train:
                test_loss = float(np.mean(losses))
                return test_loss

        best_loss = float('inf')
        self.tokens = 0 # counter used for learning rate decay

        train_loader = DataLoader(
            self.train_dataset,
            shuffle=True,
            pin_memory=True,
            batch_size=config.batch_size,
            num_workers=config.num_workers
        )
        if self.test_dataset is not None:
            test_loader = DataLoader(
                self.test_dataset,
                shuffle=True,
                pin_memory=True,
                batch_size=config.batch_size,
                num_workers=config.num_workers
            )

        for epoch in range(config.max_epochs):
            run_epoch(train_loader, is_train=True)
            if self.test_dataset is not None:
                test_loss = run_epoch(test_loader, is_train=False)

            # supports early stopping based on the test loss, or just save always if no test set is provided
            good_model = self.test_dataset is None or test_loss < best_loss
            if self.config.ckpt_path is not None and good_model:
                best_loss = test_loss
                self.save_checkpoint()
                
        

# run 

In [233]:
tconf = TrainerConfig(max_epochs=10, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(ds_X_train)*(ndigit+1),
                      num_workers=0)

In [234]:
model = GPT()

trainer = Trainer(model, train_dataset, test_dataset, tconf)
trainer.train()

number of parameters: %e 400128


epoch 1 iter 15: train loss 1.63915. lr 5.994565e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 12.06it/s]
epoch 2 iter 15: train loss 1.42369. lr 5.977303e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 12.08it/s]
epoch 3 iter 15: train loss 1.27394. lr 5.948270e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 11.95it/s]
epoch 4 iter 15: train loss 1.22703. lr 5.907582e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 11.85it/s]
epoch 5 iter 15: train loss 1.17454. lr 5.855400e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 11.67it/s]
epoch 6 iter 15: train loss 0.99136. lr 5.791929e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 11.83it/s]
epoch 7 iter 15: train loss 0.71794. lr 5.717421e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 11.67it/s]
epoch 8 iter 15: train loss 0.63615. lr 5.632171e-04: 100%|██████████████████████████████████| 16/16 [00:01<00:00, 11.

KeyboardInterrupt: 

In [199]:
ret=np.argmax(model(train_dataset[:][0])[0].detach().numpy(),axis=2)[:,-3:]

In [200]:
l=[]
for i in range(len(train_dataset)):
    l.append(train_dataset[i][1][-3:].detach().numpy())
l=np.asarray(l)

In [201]:
correct=sum(ret[:, -1]==l[:, -1])

In [202]:
correct/len(train_dataset)

1.0

In [203]:
ret=np.argmax(model(test_dataset[:][0])[0].detach().numpy(),axis=2)[:,-3:]

In [204]:
l=[]
for i in range(len(test_dataset)):
    l.append(test_dataset[i][1][-3:].detach().numpy())
l=np.asarray(l)

In [205]:
correct=sum(ret[:, -1]==l[:, -1])

In [206]:
correct/len(test_dataset)

1.0

In [ ]:
GPT().configure_optimizers(tconf)

In [67]:
x = ds_X_train 
x = Embedder(block_size,vocab_size,n_embd)(x)
B,T,C = x.size() 

print("B,T inpute size, C embedded dimension:",B,T,C)
print("dtype:",x.dtype)

B,T inpute size, C embedded dimension: 8000 6 128
dtype: torch.float32


In [138]:
Block()(x).size()

torch.Size([8000, 6, 128])

In [161]:
x = ds_X_train[1].reshape(1,-1) 

In [163]:
x.shape

(1, 6)

In [189]:
# ToDo Dataloader not compatible
# K,V,Q Layer better understanding ! multihead process.. 